In [ ]:
package_path = '../input/pytorch-image-models/pytorch-image-models-master'

import sys
sys.path.append(package_path)

DATA_DIR = '../input/cassava-leaf-disease-classification'

MODEL_DIR_0 = '../input/gpu-vit-noisearch-amp-aug-fold-0'
MODEL_DIR_1 = '../input/gpu-vit-noisearch-amp-aug-fold-1'
MODEL_DIR_2 = '../input/gpu-vit-noisearch-amp-aug-fold-2'
MODEL_DIR_3 = '../input/gpu-vit-noisearch-amp-aug-fold-3'
MODEL_DIR_4 = '../input/gpu-vit-noisearch-amp-aug-fold-4'

MODEL_DIR_01 = '../input/gpu-en-b4-ns-noisearch-amp-aug-fold-0'
MODEL_DIR_11 = '../input/gpu-en-b4-ns-noisearch-amp-aug-fold-1'
MODEL_DIR_21 = '../input/gpu-en-b4-ns-noisearch-amp-aug-fold-2'
MODEL_DIR_31 = '../input/gpu-en-b4-ns-noisearch-amp-aug-fold-3'
MODEL_DIR_41 = '../input/gpu-en-b4-ns-noisearch-amp-aug-fold-4'

MODEL_DIR_02 = '../input/gpu-seresnext-noisearch-amp-aug-fold-0'
MODEL_DIR_12 = '../input/gpu-seresnext-noisearch-amp-aug-fold-1'
MODEL_DIR_22 = '../input/gpu-seresnext-noisearch-amp-aug-fold-2'
MODEL_DIR_32 = '../input/gpu-seresnext-noisearch-amp-aug-fold-3'
MODEL_DIR_42 = '../input/gpu-seresnext-noisearch-amp-aug-fold-4'

MODEL_DIR_03 = '../input/gpu-en-b4-ns-amp-aug-fold-0'
MODEL_DIR_13 = '../input/gpu-en-b4-ns-amp-aug-fold-1'
MODEL_DIR_23 = '../input/gpu-en-b4-ns-amp-aug-fold-2'
MODEL_DIR_33 = '../input/gpu-en-b4-ns-amp-aug-fold-3'
MODEL_DIR_43 = '../input/gpu-en-b4-ns-amp-aug-fold-4'


In [ ]:
from datetime import datetime
import os
import random
import time
import warnings

import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import timm

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

In [ ]:
CFG = {
    'fold_num': 5,
    'seed': 719,
    'model_arch': 'vit_base_patch16_384',
    'img_size': 384,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 16,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # support to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 3,
    'used_epochs': [7,8,9],
    'weights': [1,1,1,1,1,1]
}

In [ ]:
EPOCHS0 = {
    0: [9,8,6,5],
    1: [5,6,4,8],
    2: [8,9,7,6],
    3: [8,7,9,6],
    4: [9,8,7,4]
         }

EPOCHS1 = {
    0: [8,9,7,6],
    1: [9,4,8,6],
    2: [9,7,8,4],
    3: [5,8,9,3],
    4: [6,7,8,9]
         }

EPOCHS2 = {
    0: [8,9,6,7],
    1: [9,8,5,6],
    2: [9,7,5,4],
    3: [5,8,9,4],
    4: [5,8,9,7]
         }

EPOCHS3 = {
    0: [8,9,7,6],
    1: [2,7,9,5],
    2: [5,6,9,7],
    3: [8,9,7,6],
    4: [4,5,8,7]
         }


In [ ]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        img  = get_img(f"{self.data_root}/{self.df.loc[index]['image_id']}")
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label:
            return img, target
        else:
            return img

In [ ]:
def get_inference_transforms_384():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)


def get_inference_transforms():
    return Compose([
            RandomResizedCrop(512, 512),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
class CassvaImgClassifierN(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.n_class = n_class
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        
        if 'vit' in model_arch:
            n_features = self.model.head.in_features
            self.model.head = nn.Identity()
        
        if 'eff' in model_arch:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Identity()
            
        if 'resnext' in model_arch:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Identity()
        
        self.layer_noise = nn.Linear(n_features, n_class)
        self.layer_pure = nn.Linear(n_features, n_class)
        

    def forward(self, x):
        x = self.model(x)
        x1 = self.layer_noise(x)
        x2 = self.layer_pure(x)
        x1 = x1 + x2
        return x1, x2

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        _, image_preds = model(imgs)
        image_preds_all += [torch.sigmoid(image_preds).detach().cpu().numpy()]
        
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
def inference_one_epochN(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)
        image_preds_all += [torch.sigmoid(image_preds).detach().cpu().numpy()]
        
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
def run_inference(fold, MODEL_DIR, model_arch, epoch_list):
    print(f'Inference fold {fold} started')


    test = pd.DataFrame()
    test['image_id'] = list(os.listdir(f'{DATA_DIR}/test_images/'))
    
    if 'vit' in model_arch:
        T = get_inference_transforms_384()
    else:
        T = get_inference_transforms()
    
    test_ds = CassavaDataset(
        test, f'{DATA_DIR}/test_images/',
        transforms=T, output_label=False)


    tst_loader = torch.utils.data.DataLoader(
        test_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )

    device = torch.device(CFG['device'])
    model = CassvaImgClassifier(model_arch, train.label.nunique()).to(device)

    tst_preds = []

    for i, epoch in enumerate(epoch_list[fold][:1]):

        model.load_state_dict(torch.load(f'{MODEL_DIR}/{model_arch}_fold_{fold}_{epoch}'))

        with torch.no_grad():
            for _ in range(CFG['tta']):
                tst_image_preds = inference_one_epoch(model, tst_loader, device)
                tst_preds += [(1/CFG['tta'])*tst_image_preds]

    tst_preds = np.sum(tst_preds, axis=0) 

    del model
    torch.cuda.empty_cache()
    return tst_preds

In [ ]:
def run_inferenceN(fold, MODEL_DIR, model_arch):
    print(f'Inference fold {fold} started')
    test = pd.DataFrame()
    test['image_id'] = list(os.listdir(f'{DATA_DIR}/test_images/'))
    test_ds = CassavaDataset(
        test, f'{DATA_DIR}/test_images/',
        transforms=get_inference_transforms(), output_label=False)


    tst_loader = torch.utils.data.DataLoader(
        test_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )

    device = torch.device(CFG['device'])
    model = CassvaImgClassifierN(model_arch, train.label.nunique()).to(device)

    tst_preds = []

    for i, epoch in enumerate(CFG['used_epochs']):

        model.load_state_dict(
            torch.load(f'{MODEL_DIR}/{model_arch}_fold_{fold}_{epoch}'))

        with torch.no_grad():
            for _ in range(CFG['tta']):
                tst_image_preds = inference_one_epochN(model, tst_loader, device)
                tst_preds += [(1/ CFG['tta'])*tst_image_preds]

    tst_preds = np.mean(tst_preds, axis=0) 

    del model
    torch.cuda.empty_cache()
    return tst_preds

In [ ]:
preds0 = run_inference(0, MODEL_DIR_0, 'vit_base_patch16_384', EPOCHS0)
preds1 = run_inference(1, MODEL_DIR_1, 'vit_base_patch16_384', EPOCHS0)
preds2 = run_inference(2, MODEL_DIR_2, 'vit_base_patch16_384', EPOCHS0)
preds3 = run_inference(3, MODEL_DIR_3, 'vit_base_patch16_384', EPOCHS0)
preds4 = run_inference(4, MODEL_DIR_4, 'vit_base_patch16_384', EPOCHS0)

# tst_preds = preds0
PRED0 = (preds0 + preds1 + preds2 + preds3 + preds4)/5

In [ ]:
preds0 = run_inference(0, MODEL_DIR_01, 'tf_efficientnet_b4_ns', EPOCHS1)
preds1 = run_inference(1, MODEL_DIR_11, 'tf_efficientnet_b4_ns', EPOCHS1)
preds2 = run_inference(2, MODEL_DIR_21, 'tf_efficientnet_b4_ns', EPOCHS1)
preds3 = run_inference(3, MODEL_DIR_31, 'tf_efficientnet_b4_ns', EPOCHS1)
preds4 = run_inference(4, MODEL_DIR_41, 'tf_efficientnet_b4_ns', EPOCHS1)

# tst_preds = preds0
PRED1 = (preds0 + preds1 + preds2 + preds3 + preds4)/5

In [ ]:
preds0 = run_inference(0, MODEL_DIR_02, 'seresnext50_32x4d', EPOCHS2)
preds1 = run_inference(1, MODEL_DIR_12, 'seresnext50_32x4d', EPOCHS2)
preds2 = run_inference(2, MODEL_DIR_22, 'seresnext50_32x4d', EPOCHS2)
preds3 = run_inference(3, MODEL_DIR_32, 'seresnext50_32x4d', EPOCHS2)
preds4 = run_inference(4, MODEL_DIR_42, 'seresnext50_32x4d', EPOCHS2)

# tst_preds = preds0
PRED2 = (preds0 + preds1 + preds2 + preds3 + preds4)/5

In [ ]:
preds0 = run_inferenceN(0, MODEL_DIR_03, 'tf_efficientnet_b4_ns',)
preds1 = run_inferenceN(1, MODEL_DIR_13, 'tf_efficientnet_b4_ns',)
preds2 = run_inferenceN(2, MODEL_DIR_23, 'tf_efficientnet_b4_ns',)
preds3 = run_inferenceN(3, MODEL_DIR_33, 'tf_efficientnet_b4_ns',)
preds4 = run_inferenceN(4, MODEL_DIR_43, 'tf_efficientnet_b4_ns')

# tst_preds = preds0
PRED3 = (preds0 + preds1 + preds2 + preds3 + preds4)/5

In [ ]:
tst_preds = (PRED0 + 2*PRED1 + PRED2 + PRED3)/5

In [ ]:
test = pd.DataFrame()
test['image_id'] = list(os.listdir(f'{DATA_DIR}/test_images/'))

test['label'] = np.argmax(tst_preds, axis=1)
test.to_csv('submission.csv', index=False)

In [ ]:
test.head()